# Fourier series

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def Periodicise(xmin, xmax):
    """Function decorator that extends a function
    by making it periodic.
    """
    def xshift(x):
        y = np.remainder(x-xmin, (xmax-xmin)) + xmin
        return y
    
    def periodicised(f):
        return lambda x: f(xshift(x))
    
    return lambda f: periodicised(f)

def Reflect(sig, about=0):
    """Function decorator that extends a function
    by reflecting it.
    """
    if sig not in (1, -1, "even", "odd"):
        raise ValueError
    if sig == "even": sig = 1
    if sig == "odd": sig = -1
        
    def inner(f, x):
        if x > about:
            return f(x)
        elif x == about:
            return f(x) if sig == 1 else 0
        else:
            return sig * f(2*about - x)
        
    def reflected(f):
        return lambda x: inner(f, x)
    
    return lambda f: reflected(f)

def myfft(ys):
    """Our FFT convention that we will use in this course.
    Note that conventions vary between packages and authors
    over the normalisation.
    For this course, we want our FFTs to be independent of
    the sample rate, so we'll normalise by multiplying by 2/nx.
    """
    return np.fft.fft(ys, norm=None) * (2/len(ys))

def anbn(ys):
    """Return two arrays, the cosine and sine coefficients.
    Note that the 0-frequency term in the series (in this course's
    convention) is a0/2."""
    ft = myfft(ys)
    ans = np.real(ft)
    bns = -np.imag(ft)
    return np.array([ans, bns])

def partial_fourier_sum(fourier_coeffs, period, nterms=5):
    an, bn = fourier_coeffs
    def Sn(x):
        psum = an[0]/2
        for i in range(1, nterms+1):
            psum += an[i]*np.cos(i*2*np.pi*x / period)
            psum += bn[i]*np.sin(i*2*np.pi*x / period)
        return psum
    return Sn

In [ ]:
# The domain (we'll go from 0 to xmax, and may need
# to periodicise or translate our function if it was
# originally defined on a different domain)
nx = 256
xmax = 2*np.pi
dx = xmax/nx

# Angular wavenumbers (or frequency). Angular, so 2pis in numerator.
# The wavenumbers in principle go up to 2pi/dx,
# but actually only half of these carry any physical information,
# up to the Nyquist frequency of 2pi / (2dx).
nyquist = np.pi / dx 
ks = np.linspace(0, 2*nyquist, nx+1)[0:-1]

# For looking at Fourier series coefficients, we'll multiply by L/(2pi)
# to get the integer wavenumbers.
ns = ks * xmax / (2*np.pi)

# Question 1

In [ ]:
nx = 256
period = 2

@Periodicise(-1, 1)
# @Reflect("odd", 0)
def f_unv(x):
    return (x**2 - 1)**2 # Q1
f = np.vectorize(f_unv, otypes=[float])
# Note need to specify float explicitly when vectorizing
# (otherwise, type determined automatically from the first output)

# We'll take the domain to be from 0 to 2 instead of -1 to 1.
# Doesn't matter, since the function is periodic anyway.
xs = np.linspace(0, period, nx+1)[:-1]
ys = f(xs) # Why does this get floored?
# Plot beyond our domain (just for demonstration purposes)
xps = np.linspace(-2, 2, 256)
yps = np.array([f(x) for x in xps])

fig, axs = plt.subplots(2, 1, figsize=[14, 8])
# ax.set(xlim=[0, xmax])
axs[0].grid()
axs[0].plot(xs, ys, "k",
        xps, yps, 'k--')

yft = myfft(ys)
fourier_coeffs = anbn(ys)

axs[1].set(title="Fourier coefficients")
axs[1].legend(['cosine', 'sine'])
axs[1].set(xlim=[0, 12])
axs[1].grid()
axs[1].plot(ns, fourier_coeffs[0], 'ko',
            ns, fourier_coeffs[1], 'rx')

plt.show()

fig, axs = plt.subplots(figsize=[14, 5])
# ax.set(xlim=[0, xmax])
axs.set(title="Partial sums of the Fourier series")
axs.grid()
axs.plot(xps, yps, "k",
         xps, partial_fourier_sum(fourier_coeffs, period, 1)(xps), 'r--')
plt.show()

## Question 2: Half-range series

In [ ]:
nx = 256
period = 2*np.pi

@Periodicise(-np.pi, np.pi)
@Reflect("odd", 0)
def f(x):
    return x**2

f = np.vectorize(f, otypes=[float])

xs = np.linspace(0, period, nx+1)[:-1]
ys = f(xs) # Why does this get floored?
# Plot beyond our domain (just for demonstration purposes)
xps = np.linspace(-2*np.pi, 2*np.pi, 256)
yps = f(xps)
yft = myfft(ys)
fourier_coeffs = anbn(ys)

fig, axs = plt.subplots(2, 1, figsize=[14, 8])
axs[0].grid()
axs[0].plot(xs, ys, "k",
            xps, yps, 'k:',
            xps, partial_fourier_sum(fourier_coeffs, period, 10)(xps), 'r--')


axs[1].set(title="Fourier coefficients")
axs[1].legend(['cosine', 'sine'])
axs[1].set(xlim=[0, 12])
axs[1].grid()
axs[1].plot(ns, fourier_coeffs[0], 'ko',
            ns, fourier_coeffs[1], 'rx')

plt.show()

## Question 3: Series summation

In [ ]:
nx = 256
period = 2*np.pi

@Periodicise(-np.pi, np.pi)
def f(x):
    return np.exp(x)

f = np.vectorize(f, otypes=[float])

xs = np.linspace(0, period, nx+1)[:-1]
ys = f(xs) # Why does this get floored?
# Plot beyond our domain (just for demonstration purposes)
xps = np.linspace(-2*np.pi, 2*np.pi, 256)
yps = f(xps)
yft = myfft(ys)
fourier_coeffs = anbn(ys)

fig, axs = plt.subplots(2, 1, figsize=[14, 8])
axs[0].grid()
axs[0].plot(xs, ys, "k",
            xps, yps, 'k:',
            xps, partial_fourier_sum(fourier_coeffs, period, 10)(xps), 'r--')


axs[1].set(title="Fourier coefficients")
axs[1].legend(['cosine', 'sine'])
axs[1].set(xlim=[0, 12])
axs[1].grid()
axs[1].plot(ns, fourier_coeffs[0], 'ko',
            ns, fourier_coeffs[1], 'rx')

plt.show()

## Question 5: Square wave

In [ ]:
@Periodicise(0, 2*np.pi)
def f_unv(x):
    return (1 + np.sign(np.sin(x))) / 2 # square wave - Q5

f = np.vectorize(f_unv, otypes=[float])
nx = 256
xmax = 2*np.pi
dx = xs[1] - xs[0]
nyquist = np.pi / dx 
ks = np.linspace(0, 2*nyquist, nx+1)[0:-1]
ns = ks * xmax / (2*np.pi)

xs = np.linspace(0, xmax, nx+1)[:-1]
ys = f(xs)
yft = myfft(ys)
fourier_coeffs = anbn(ys)

fig, axs = plt.subplots(figsize=[14, 5])
# ax.set(xlim=[0, xmax])
axs.grid()
axs.plot(xs, ys, "k",
         xs, partial_fourier_sum(fourier_coeffs, xmax, 20)(xs), 'r--')
plt.show()

## Bonus: Audio clipping and distortion


In [ ]:
nx = 256
period = 1

def Clipping(maxamp=1):
    """Function decorator that returns a
    clipped version of a function.
    """
    def inner(f, x):
        y = f(x)
        return max(-1, min(1, y))

    def clipped(f):
        return lambda x: inner(f, x)

    return lambda f: clipped(f)

@interact(amp=widgets.FloatSlider(min=0, max=1.5, value=0.4))
def audio_clipping_demo(amp):
    def f_unv(x):
        return amp * np.sin(2*np.pi*x)

    f_noclip = np.vectorize(f_unv, otypes=[float])
    f = np.vectorize(Clipping()(f_noclip))

    xs = np.linspace(0, period, nx+1)[:-1]
    ys = f(xs) 
    xps = np.linspace(0, 2*period, 256)
    yps = np.array([f_noclip(x) for x in xps])
    yft = myfft(ys)
    fourier_coeffs = anbn(ys)

    fig, axs = plt.subplots(1, 2, figsize=[14, 6])
    axs[0].set(xlim=[0, 2*period], ylim=[-1.6, 1.6])
    axs[0].grid()
    axs[0].plot(xs, ys, "k",
                xps, yps, 'k--',
                xps, partial_fourier_sum(fourier_coeffs, period, 12)(xps), 'r--')


    axs[1].set(title="Fourier coefficients (abs)")
    axs[1].set(xlim=[0, 12], ylim=[-.1, 1.3])
    axs[1].grid()
    axs[1].plot(ns[1::2], (fourier_coeffs[1][1::2]), 'r-x')

    plt.show()
